In [23]:
import torch
model = torch.hub.load('mateuszbuda/brain-segmentation-pytorch', 'unet',
    in_channels=3, out_channels=1, init_features=32, pretrained=True,trust_repo=True)

Using cache found in C:\Users\sujal/.cache\torch\hub\mateuszbuda_brain-segmentation-pytorch_master


In [24]:
import urllib
url, filename = ("https://github.com/mateuszbuda/brain-segmentation-pytorch/raw/master/assets/TCGA_CS_4944.png", "TCGA_CS_4944.png")
try: urllib.URLopener().retrieve(url, filename)
except: urllib.request.urlretrieve(url, filename)

In [31]:
import torchvision
from torchvision.transforms import transforms
import numpy as np
from PIL import Image
import torch


# Load the image
input_image = Image.open(filename)

# Convert the image to a NumPy array for calculating mean and std
input_image_np = np.array(input_image)

# Calculate mean and std (for each channel)
m, s = np.mean(input_image_np, axis=(0, 1)), np.std(input_image_np, axis=(0, 1))

# Preprocessing pipeline
preprocess = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=m / 255.0, std=s / 255.0),  # Normalize expects values between 0 and 1
])

# Preprocess the image
input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(0)  # Add batch dimension

# Check if GPU is available and move data/model to GPU if so
if torch.cuda.is_available():
    input_batch = input_batch.to('cuda')
    model = model.to('cuda')

# Inference
with torch.no_grad():
    output = model(input_batch)

# Print the rounded output
print(torch.round(output[0]))


AttributeError: partially initialized module 'torchvision' has no attribute 'extension' (most likely due to a circular import)

In [3]:
from typing import Optional

import requests
from batchgenerators.utilities.file_and_folder_operations import *
from time import time
from nnunetv2.model_sharing.model_import import install_model_from_zip_file
from nnunetv2.paths import nnUNet_results
from tqdm import tqdm

nnUNet_raw is not defined and nnU-Net can only be used on data for which preprocessed files are already present on your system. nnU-Net cannot be used for experiment planning and preprocessing like this. If this is not intended, please read documentation/setting_up_paths.md for information on how to set this up properly.
nnUNet_preprocessed is not defined and nnU-Net can not be used for preprocessing or training. If this is not intended, please read documentation/setting_up_paths.md for information on how to set this up.
nnUNet_results is not defined and nnU-Net cannot be used for training or inference. If this is not intended behavior, please read documentation/setting_up_paths.md for information on how to set this up.


In [4]:
def download_and_install_from_url(url):
    assert nnUNet_results is not None, "Cannot install model because network_training_output_dir is not " \
                                                    "set (RESULTS_FOLDER missing as environment variable, see " \
                                                    "Installation instructions)"
    print('Downloading pretrained model from url:', url)
    import http.client
    http.client.HTTPConnection._http_vsn = 10
    http.client.HTTPConnection._http_vsn_str = 'HTTP/1.0'

    import os
    home = os.path.expanduser('~')
    random_number = int(time() * 1e7)
    tempfile = join(home, f'.nnunetdownload_{str(random_number)}')

    try:
        download_file(url=url, local_filename=tempfile, chunk_size=8192 * 16)
        print("Download finished. Extracting...")
        install_model_from_zip_file(tempfile)
        print("Done")
    except Exception as e:
        raise e
    finally:
        if isfile(tempfile):
            os.remove(tempfile)

In [5]:
def download_file(url: str, local_filename: str, chunk_size: Optional[int] = 8192 * 16) -> str:
    # borrowed from https://stackoverflow.com/questions/16694907/download-large-file-in-python-with-requests
    # NOTE the stream=True parameter below
    with requests.get(url, stream=True, timeout=100) as r:
        r.raise_for_status()
        with tqdm.wrapattr(open(local_filename, 'wb'), "write", total=int(r.headers.get("Content-Length"))) as f:
            for chunk in r.iter_content(chunk_size=chunk_size):
                f.write(chunk)
    return local_filename

In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class UNet(nn.Module):
    def __init__(self, in_channels=3, out_channels=1):
        super(UNet, self).__init__()
        self.encoder1 = self.conv_block(in_channels, 64)
        self.encoder2 = self.conv_block(64, 128)
        self.encoder3 = self.conv_block(128, 256)
        self.encoder4 = self.conv_block(256, 512)
        self.bottleneck = self.conv_block(512, 1024)
        
        self.decoder4 = self.upconv_block(1024, 512)
        self.decoder3 = self.upconv_block(512, 256)
        self.decoder2 = self.upconv_block(256, 128)
        self.decoder1 = self.upconv_block(128, 64)
        
        self.final_conv = nn.Conv2d(64, out_channels, kernel_size=1)

    def conv_block(self, in_channels, out_channels):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True)
        )

    def upconv_block(self, in_channels, out_channels):
        return nn.Sequential(
            nn.ConvTranspose2d(in_channels, out_channels, kernel_size=2, stride=2),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        enc1 = self.encoder1(x)
        enc2 = self.encoder2(F.max_pool2d(enc1, kernel_size=2))
        enc3 = self.encoder3(F.max_pool2d(enc2, kernel_size=2))
        enc4 = self.encoder4(F.max_pool2d(enc3, kernel_size=2))
        bottleneck = self.bottleneck(F.max_pool2d(enc4, kernel_size=2))
        dec4 = self.decoder4(bottleneck)
        dec4 = torch.cat((dec4, enc4), dim=1)
        dec4 = self.decoder3(dec4)
        dec4 = torch.cat((dec4, enc3), dim=1)
        dec3 = self.decoder2(dec4)
        dec3 = torch.cat((dec3, enc2), dim=1)
        dec2 = self.decoder1(dec3)
        dec2 = torch.cat((dec2, enc1), dim=1)
        return self.final_conv(dec2)

# Initialize the model and load the weights
model = UNet(in_channels=3, out_channels=1)  # Adjust channels as needed
model.load_state_dict(torch.load('3channel50.pt', map_location=torch.device('cpu')),strict=False)
model.eval()


C:\Users\sujal\AppData\Local\Temp\ipykernel_2332\2642201277.py:53: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('3channel50.pt', map_locati

UNet(
  (encoder1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
  )
  (encoder2): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
  )
  (encoder3): Sequential(
    (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
  )
  (encoder4): Sequential(
    (0): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
  )
  (bottleneck): Sequential(
    (0): Con

In [16]:
import cv2
def preprocess_image(image_path):
    # Load image using OpenCV (change this if you use PIL)
    image = cv2.imread(image_path)
    
    # Resize the image if necessary
    image = cv2.resize(image, (256, 256))  # Change to your required input size
    
    # Convert BGR to RGB
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Convert to tensor
    image_tensor = transforms.ToTensor()(image)
    
    # Add batch dimension
    image_tensor = image_tensor.unsqueeze(0)  # Shape: (1, 3, 256, 256)
    
    return image_tensor

# Function to postprocess the output
def postprocess_output(output_tensor):
    # Convert tensor to numpy array
    output_array = output_tensor.squeeze(0).detach().numpy()  # Shape: (1, 256, 256)
    
    # Binarize the output (thresholding)
    output_array = (output_array > 0.5).astype(np.uint8)  # Adjust threshold as necessary
    
    return output_array * 255  # Scale to [0, 255] for visualization

# Load and preprocess the image
input_image_path = r'C:\Users\sujal\PycharmProjects\DSU\dicom_communication\0.dcm'  # Replace with your image path
input_tensor = preprocess_image(input_image_path)

# Perform inference
with torch.no_grad():  # Disable gradient calculation
    output_tensor = model(input_tensor)

# Postprocess the output
segmented_image = postprocess_output(output_tensor)

# Display or save the segmented image
segmented_image_pil = Image.fromarray(segmented_image[0])  # Convert to PIL Image
segmented_image_pil.show()  # Show the image

ImportError: DLL load failed while importing _multiarray_umath: The specified module could not be found.

ImportError: numpy._core.multiarray failed to import